In [1]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import scipy as sp
import missingno as msno

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('poster', rc={'font.size':35,
                              'axes.titlesize':50,
                              'axes.labelsize':35})

#machine learning
import category_encoders as ce
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.preprocessing import StandardScaler, Normalizer, LabelEncoder, PolynomialFeatures
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold, train_test_split, cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [2]:
train_data_path = 'C:/Users/Schindler/Documents/ProgrammingFun/GOSSIS_kaggle/training_v2.csv'
unlabeled_data_path = 'C:/Users/Schindler/Documents/ProgrammingFun/GOSSIS_kaggle/unlabeled.csv'

In [3]:
train_supplied = pd.read_csv(train_data_path)
train_supplied = pd.DataFrame(data = train_supplied)

print('Original data shape:\n', train_supplied.shape, '\n')
print('Group value counts:\n', train_supplied['hospital_death'].value_counts(), '\n')

#remove columns with meta data (e.g. unique identifiers) and/or zero variance
meta_cols = ['encounter_id', 'patient_id', 'readmission_status']
train_supplied = train_supplied.drop(columns = meta_cols)

#convert negative numbers to nans 
cols_with_zeros = ['pre_icu_los_days', 'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob']
train_supplied.loc[~(train_supplied['pre_icu_los_days'] > 0), 'pre_icu_los_days']=np.nan
train_supplied.loc[~(train_supplied['apache_4a_hospital_death_prob'] > 0), 'apache_4a_hospital_death_prob']=np.nan
train_supplied.loc[~(train_supplied['apache_4a_icu_death_prob'] > 0), 'apache_4a_icu_death_prob']=np.nan

train_supplied.head()

Original data shape:
 (91713, 186) 

Group value counts:
 0    83798
1     7915
Name: hospital_death, dtype: int64 



,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,118,0,68.0,22.73,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,73.9,2.3,113.0,502.01,0,0.0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0.0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0.0,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,122.0,64.0,131.0,73.0,131.0,73.0,39.9,37.2,NaN,NaN,68.0,63.0,68.0,63.0,119.0,108.0,NaN,NaN,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,NaN,NaN,131.0,115.0,131.0,115.0,39.5,37.5,2.3,2.3,0.4,0.4,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,NaN,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,0.05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,81,0,77.0,27.42,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,70.2,NaN,108.0,203.01,0,0.0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0.0,

### explore nans and create related features 

In [4]:
#explore missing data 
data = train_supplied

na_0_mean = (data[data['hospital_death'] == 0].isna().sum() / data[data['hospital_death'] == 0].shape[0] * 100).mean()
na_1_mean = (data[data['hospital_death'] == 1].isna().sum() / data[data['hospital_death'] == 1].shape[0] * 100).mean()
print(na_0_mean, na_1_mean)

#there is a bit more missing data for the entries that lived (not surprizing as you get more tests the poorer health)
#get count of missing data per entry
train_supplied['na_count'] = train_supplied.isna().sum(axis=1)

34.78926750341732 29.67444397267414


In [5]:
#are there parameters that have more missing values for yes vs no death
data = train_supplied

na_0 = (data[data['hospital_death'] == 0].isna().sum() / data[data['hospital_death'] == 0].shape[0]) * 100
na_1 = (data[data['hospital_death'] == 1].isna().sum() / data[data['hospital_death'] == 1].shape[0]) * 100
na_0_diff = data[data['hospital_death'] == 0].shape[0] - data[data['hospital_death'] == 0].isna().sum()
na_1_diff = data[data['hospital_death'] == 1].shape[0] - data[data['hospital_death'] == 1].isna().sum()
data_na_perc = pd.DataFrame(data=[na_0, na_1, na_0_diff, na_1_diff])
data_na_perc = data_na_perc.T.sort_values(by=0, ascending=False)
data_na_perc.columns = ['na_0', 'na_1', 'na_0_diff', 'na_1_diff']
data_na_perc['diff'] = data_na_perc['na_0'] - data_na_perc['na_1']
data_na_perc = data_na_perc.sort_values(by='diff', ascending=False)

#get count for each entry of how many missing for params that have at least 25% difference in missing between yes and no death
param_25_diff = data_na_perc[data_na_perc['diff'] > 25].index
train_supplied['param_25_diff_count'] = train_supplied[param_25_diff].isna().sum(axis=1)

In [6]:
#remove features that have more than 50% missing data
print(train_supplied.shape)
train_missing = (train_supplied.isnull().sum() / len(train_supplied)).sort_values(ascending = False)
train_missing = train_missing.index[train_missing > 0.50]
print('There are %d columns with more than 50%% missing values' % len(train_missing))
train_supplied_missing = train_supplied.copy().drop(columns = train_missing)
print(train_supplied_missing.shape)

(91713, 185)
There are 74 columns with more than 50% missing values
(91713, 111)


### feature mapping

In [7]:
def age_mapping(age):
    
    # based off of APACHE III scoring system
    
    score = np.nan
    
    if age == np.nan:
        score = np.nan
    elif age <= 44:
        score = 0
    elif 45 <= age <=59:
        score = 5
    elif 60 <= age <=64:
        score = 11
    elif 65 <= age <=69:
        score = 13
    elif 70 <= age <=74:
        score = 16
    elif 75 <= age <=84:
        score = 17
    elif age >= 85:
        score = 24
        
    return score
    
train_supplied_missing['age_score'] = train_supplied_missing.apply(lambda row : age_mapping(row['age']), axis = 1)

In [8]:
def chronic_health_mapping(row, health_conds):
    
    df_int = row[health_conds]
    
    # based off of info from APACHE II scoring system
    
    score = 0
    
    if df_int.isna().sum() == 9: #all nans
        score = np.nan
        
    if df_int['aids'] > 0:
        score += 23
    if df_int['cirrhosis'] > 0:
        score += 4
    if df_int['diabetes_mellitus'] > 0:
        score += 2 
    if df_int['hepatic_failure'] > 0:
        score += 16
    if df_int['immunosuppression'] > 0:
        score += 10
    if df_int['leukemia'] > 0:
        score += 10
    if df_int['lymphoma'] > 0:
        score += 13
    if df_int['solid_tumor_with_metastasis'] > 0:
        score += 11   
        
    return score

health_conds = ['aids', 'cirrhosis', 'diabetes_mellitus', 'hepatic_failure',
       'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis']

train_supplied_missing['chronic_health_score'] = train_supplied_missing.apply(lambda row : chronic_health_mapping(row, health_conds), axis = 1)

In [9]:
#create feature that is count of disease diagnosis
diseases = ['aids', 'cirrhosis', 'diabetes_mellitus', 'hepatic_failure',
       'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis']

train_supplied_missing['disease_count'] = train_supplied_missing[diseases].dropna().sum(axis=1)

def elective_surgery_mapping(elective_surgery, disease_count):
    
    # based off of APACHE II scoring system
    
    score = np.nan
    
    if disease_count == np.nan:
        score = np.nan
    
    if disease_count == 0:
        score = 0
    
    if disease_count > 0:
        if elective_surgery == 1:
            score = 2
        if elective_surgery == 0:
            score = 5
        
    return score
    
train_supplied_missing['elective_surgery_score'] = train_supplied_missing.apply(lambda row : elective_surgery_mapping(row['elective_surgery'], row['disease_count']), axis = 1)

In [10]:
def bun_mapping(bun_apache):
    
    # based off of APACHE III scoring system
    
    score = np.nan
    
    if bun_apache == np.nan:
        score = np.nan
    elif bun_apache <= 6.1:
        score = 0
    elif 6.2 <= bun_apache <= 7.1:
        score = 2
    elif 7.2 <= bun_apache <= 14.3:
        score = 7
    elif 14.4 <= bun_apache <= 28.5:
        score = 11
    elif bun_apache >= 28.6:
        score = 12
        
    return score
    
train_supplied_missing['bun_score'] = train_supplied_missing.apply(lambda row : bun_mapping(row['bun_apache']), axis = 1)

In [11]:
def creatinine_mapping(creatinine_apache, arf_apache):
    
    # based off of APACHE III scoring system
    
    score = np.nan
    
    if creatinine_apache == np.nan:
        score = np.nan
    
    elif arf_apache == 1:
        if creatinine_apache <= 0.40:
            score = 3
        elif 0.41 <= creatinine_apache <= 1.40:
            score = 0
        elif 1.41 <= creatinine_apache <= 1.94:
            score = 4
        elif creatinine_apache >= 1.95:
            score = 7   
        
    else: #assume nan is same as no arf
        if creatinine_apache <= 1.40:
            score = 0
        elif creatinine_apache >= 1.41:
            score = 10
        
    return score
    
train_supplied_missing['creatinine_score'] = train_supplied_missing.apply(lambda row : creatinine_mapping(row['creatinine_apache'], row['arf_apache']), axis = 1)

In [12]:
def GCS_mapping(row, gcs_params):
    
    
    df_int = row[gcs_params]
    
    # based off of info from APACHE III scoring system and google search
    
    score = 0
    
    if df_int.isna().sum() == 4: #all nans
        score = np.nan

    elif df_int['gcs_unable_apache'] == 1: #under sedation so cannot give responses
            score = 2
    
    elif df_int['gcs_unable_apache'] == 0:
        score = df_int[['gcs_eyes_apache', 'gcs_motor_apache', 'gcs_verbal_apache']].sum()
                
    return score

gcs_params = ['gcs_eyes_apache',
       'gcs_motor_apache', 'gcs_unable_apache', 'gcs_verbal_apache']

train_supplied_missing['GCS_score'] = train_supplied_missing.apply(lambda row : GCS_mapping(row, gcs_params), axis = 1)

In [13]:
def glucose_mapping(glucose_apache):
    
    # based off of APACHE III scoring system
    
    score = np.nan
    
    if glucose_apache == np.nan:
        score = np.nan
    elif glucose_apache <= 39:
        score = 8
    elif 39.1 <= glucose_apache <= 59:
        score = 7
    elif 59.1 <= glucose_apache <= 199:
        score = 0
    elif 199.1 <= glucose_apache <= 349:
        score = 3
    elif glucose_apache >= 349.1:
        score = 5
        
    return score
    
train_supplied_missing['glucose_score'] = train_supplied_missing.apply(lambda row : glucose_mapping(row['glucose_apache']), axis = 1)

In [14]:
def heart_rate_mapping(heart_rate_apache):
    
    # based off of APACHE III scoring system
    
    score = np.nan
    
    if heart_rate_apache == np.nan:
        score = np.nan
    elif heart_rate_apache <= 39:
        score = 8
    elif 40 <= heart_rate_apache <= 49:
        score = 5
    elif 50 <= heart_rate_apache <= 99:
        score = 0
    elif 100 <= heart_rate_apache <= 109:
        score = 1
    elif 110 <= heart_rate_apache <= 119:
        score = 5
    elif 120 <= heart_rate_apache <= 139:
        score = 7
    elif 140 <= heart_rate_apache <= 154:
        score = 13
    elif heart_rate_apache >= 155:
        score = 17
        
    return score
    
train_supplied_missing['heart_rate_score'] = train_supplied_missing.apply(lambda row : heart_rate_mapping(row['heart_rate_apache']), axis = 1)

In [15]:
def hematocrit_mapping(hematocrit_apache):
    
    # based off of APACHE III scoring system
    
    score = np.nan
    
    if hematocrit_apache == np.nan:
        score = np.nan
    elif hematocrit_apache <= 40.9:
        score = 3
    elif 41 <= hematocrit_apache <= 49:
        score = 0
    elif hematocrit_apache >= 49.1:
        score = 3
        
    return score
    
train_supplied_missing['hematocrit_score'] = train_supplied_missing.apply(lambda row : hematocrit_mapping(row['hematocrit_apache']), axis = 1)

In [16]:
def map_mapping(map_apache):
    
    # based off of APACHE III scoring system
    
    score = np.nan
    
    if map_apache == np.nan:
        score = np.nan
    elif map_apache <= 39:
        score = 8
    elif 40 <= map_apache <= 49:
        score = 5
    elif 50 <= map_apache <= 99:
        score = 0
    elif 100 <= map_apache <= 109:
        score = 1
    elif 110 <= map_apache <= 119:
        score = 5
    elif 120 <= map_apache <= 139:
        score = 7
    elif 140 <= map_apache <= 154:
        score = 13
    elif map_apache >= 155:
        score = 17
        
    return score
    
train_supplied_missing['map_score'] = train_supplied_missing.apply(lambda row : map_mapping(row['map_apache']), axis = 1)

In [17]:
def resprate_mapping(resprate_apache, ventilated_apache):
    
    # based off of APACHE III scoring system
    
    score = np.nan
    
    if resprate_apache == np.nan:
        score = np.nan
    
    else:
        if ventilated_apache == 1: #as per scoring instructions
            if resprate_apache <= 6:
                score = 17
            elif 6.1 <= resprate_apache <= 12:
                score = 0
            elif 12.1 <= resprate_apache <= 24:
                score = 0
            elif 24.1 <= resprate_apache <= 34:
                score = 6
            elif 34.1 <= resprate_apache <= 39:
                score = 9
            elif 39.1 <= resprate_apache <= 49:
                score = 11
            elif resprate_apache >= 49.1:
                score = 18
        else:
            if resprate_apache <= 5:
                score = 17
            elif 5.1 <= resprate_apache <= 11:
                score = 8
            elif 11.1 <= resprate_apache <= 13:
                score = 7
            elif 13.1 <= resprate_apache <= 24:
                score = 0
            elif 24.1 <= resprate_apache <= 34:
                score = 6
            elif 34.1 <= resprate_apache <= 39:
                score = 9
            elif 39.1 <= resprate_apache <= 49:
                score = 11
            elif resprate_apache >= 49.1:
                score = 18            

    return score
    
train_supplied_missing['resprate_score'] = train_supplied_missing.apply(lambda row : resprate_mapping(row['resprate_apache'], row['ventilated_apache']), axis = 1)

In [18]:
def sodium_mapping(sodium_apache):
    
    # based off of APACHE III scoring system
    
    score = np.nan
    
    if sodium_apache == np.nan:
        score = np.nan
    elif sodium_apache <= 134:
        score = 1
    elif 134.1 <= sodium_apache <= 154:
        score = 0
    elif sodium_apache >= 154.1:
        score = 1
        
    return score
    
train_supplied_missing['sodium_score'] = train_supplied_missing.apply(lambda row : sodium_mapping(row['sodium_apache']), axis = 1)

In [20]:
def temp_mapping(temp_apache):
    
    # based off of APACHE III scoring system
    
    score = np.nan
    
    if temp_apache == np.nan:
        score = np.nan
    elif temp_apache <= 32.99:
        score = 28
    elif 33 <= temp_apache <= 33.49:
        score = 16
    elif 33.5 <= temp_apache <= 33.99:
        score = 13
    elif 34 <= temp_apache <= 34.99:
        score = 8        
    elif 35 <= temp_apache <= 35.99:
        score = 2        
    elif 36 <= temp_apache <= 39.99:
        score = 0        
    elif temp_apache >= 40:
        score = 4
        
    return score
    
train_supplied_missing['temp_score'] = train_supplied_missing.apply(lambda row : temp_mapping(row['temp_apache']), axis = 1)

In [21]:
def wbc_mapping(wbc_apache):
    
    # based off of APACHE III scoring system
    
    score = np.nan
    
    if wbc_apache == np.nan:
        score = np.nan
    elif wbc_apache <= 0.99:
        score = 19
    elif 1 <= wbc_apache <= 2.99:
        score = 5
    elif 3 <= wbc_apache <= 19.99:
        score = 0
    elif 20 <= wbc_apache <= 24.99:
        score = 1              
    elif wbc_apache >= 25:
        score = 5
        
    return score
    
train_supplied_missing['wbc_score'] = train_supplied_missing.apply(lambda row : wbc_mapping(row['wbc_apache']), axis = 1)

In [22]:
def d1_calcium_mapping(d1_calcium_max, d1_calcium_min):
    
    # based off of APACHE III scoring system
    
    score = np.nan
    
    calcium_average = (d1_calcium_max + d1_calcium_min) / 2
    
    if d1_calcium_max == np.nan:
        score = np.nan
    elif calcium_average <= 8.2:
        score = 3
    elif 8.3 <= calcium_average <= 10.3:
        score = 0           
    elif calcium_average >= 10.4:
        score = 3
        
    return score
    
train_supplied_missing['d1_calcium_score'] = train_supplied_missing.apply(lambda row : d1_calcium_mapping(row['d1_calcium_max'], row['d1_calcium_min']), axis = 1)

In [23]:
def d1_platlet_mapping(d1_platelets_max, d1_platelets_min):
    
    # based off of APACHE III scoring system
    
    score = np.nan
    
    platlet_average = (d1_platelets_max + d1_platelets_min) / 2
    
    if d1_platelets_max == np.nan:
        score = np.nan
    elif platlet_average <= 20:
        score = 15
    elif 20.1 <= platlet_average <= 50:
        score = 7    
    elif 50.1 <= platlet_average <= 100:
        score = 3    
    elif 100.1 <= platlet_average <= 150:
        score = 1           
    elif 150.1 <= platlet_average <= 400:
        score = 0  
    elif 400.1 <= platlet_average <= 450:
        score = 1   
    elif platlet_average >= 450.1:
        score = 3
        
    return score
    
train_supplied_missing['d1_platlet_score'] = train_supplied_missing.apply(lambda row : d1_platlet_mapping(row['d1_platelets_max'], row['d1_platelets_min']), axis = 1)

In [24]:
def d1_potassium_mapping(d1_potassium_max, d1_potassium_min):
    
    # based off of APACHE III scoring system
    
    score = np.nan
    
    potassium_average = (d1_potassium_max + d1_potassium_min) / 2
    
    if d1_potassium_max == np.nan:
        score = np.nan
    elif potassium_average <= 2.4:
        score = 5 
    elif 2.5 <= potassium_average <= 3:
        score = 3 
    elif 3.1 <= potassium_average <= 3.4:
        score = 1    
    elif 3.5 <= potassium_average <= 5.1:
        score = 0  
    elif 5.2 <= potassium_average <= 6:
        score = 1 
    elif 6.1 <= potassium_average <= 7:
        score = 3    
    elif potassium_average >= 7.1:
        score = 5
        
    return score
    
train_supplied_missing['d1_potassium_score'] = train_supplied_missing.apply(lambda row : d1_potassium_mapping(row['d1_potassium_max'], row['d1_potassium_min']), axis = 1)

### encode categorical variables

In [25]:
#convert categorical columns that are ints to categorical dtype
train_supplied_missing['hospital_id'] = train_supplied_missing['hospital_id'].astype('category')
train_supplied_missing['icu_id'] = train_supplied_missing['icu_id'].astype('category')

#encode categorical variables
cat_cols = ['hospital_id', 'icu_id', 'ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 
            'icu_type', 'apache_2_diagnosis', 'apache_3j_diagnosis', 'apache_3j_bodysystem', 'apache_2_bodysystem']

#encode categorical variables using targetencoder - will need to retrain on fianl feature set
data_cat_train = train_supplied_missing.copy()

Y_train = data_cat_train['hospital_death']
X_train = data_cat_train

# use target encoding to encode two categorical features
enc = ce.TargetEncoder(cols=cat_cols, handle_missing='return_nan', handle_unknown='return_nan').fit(X_train, Y_train)
train_supplied_missing_catenc = enc.transform(X_train)
print(train_supplied_missing_catenc.shape)
train_supplied_missing_catenc.head()

(91713, 129)


,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,na_count,param_25_diff_count,age_score,chronic_health_score,disease_count,elective_surgery_score,bun_score,creatinine_score,GCS_score,glucose_score,heart_rate_score,hematocrit_score,map_score,resprate_score,sodium_score,temp_score,wbc_score,d1_calcium_score,d1_platlet_score,d1_potassium_score
0,0.070621,0,68.0,22.73,0,0.087262,0.084417,180.3,0.138796,0.134136,0.027242,0.085168,0.060205,0.541667,73.9,0.157922,0.104856,0,0.0,31.0,2.51,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0.0,40.0,36.0,134.0,39.3,0.0,14.1,68.0,37.0,68.0,37.0,119.0,72.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,131.0,73.0,131.0,73.0,39.9,37.2,68.0,63.0,68.0,63.0,119.0,108.0,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,131.0,115.0,131.0,115.0,39.5,37.5,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,0.10,0.05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.157922,0.108383,60,13,13.0,2,1.0,5.0,12.0,10.0,13.0,0.0,5.0,3.0,5.0,9.0,1.0,0.0,0.0,3.0,0.0,0.0
1,0.043103,0,77.0,27.42,0,0.087262,0.088373,160.0,0.138796,0.134136,0.043103,0.085168,0.087495,0.927778,70.2,0.197111,0.197111,0,0.0,9.0,0.56,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0.0,46.0,33.0,145.0,35.1,1.0,12.7,95.0,31.0,95.0,31.0,118.0,72.0,120.0,38.0,120.0,38.0,32.0,12.0,100.0,70.0,159.0,67.0,159.0,67.0,36.3,35.1,61.0,48.0,61.0,48.0,114.0,100.0,85.0,57.0,85.0,57.0,31.0,28.0,95.0,70.0,95.0,71.0,95.0,71.0,36.3,36.3,11.0,9.0,8.6,8.0,0.71,0.56,145.0,128.0,27.0,26.0,11.3,11.1,36.9,36.1,557.0,487.0,4.2,3.8,145.0,145.0,23.3,12.7,0.47,0.29,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.112068,0.112068,19,0,17.0,2,1.0,5.0,7.0,0.0,5.0,0.0,7.0,3.0,5.0,6.0,0.0,2.0,0.0,0.0,3.0,0.0
2,0.070621,0,25.0,31.95,0,0.087262,0.088373,172.7,0.087306,0.086385,0.067093,0.085168,0.087495,0.000694,95.3,0.008493,0.007727,0,0.0,NaN,NaN,3.0,6.0,0.0,5.0,NaN,102.0,NaN,0.0,68.0,37.0,NaN,36.7,0.0,NaN,88.0,48.0,88.0,48.0,96.0,68.0,102.0,68.0,102.0,68.0,21.0,8.0,98.0,91.0,148.0,105.0,148.0,105.0,37.0,36.7,88.0,58.0,88.0,58.0,96.0,78.0,91.0,83.0,91.0,83.0,20.0,16.0,98.0,91.0,148.0,124.0,148.0,124.0,36.7,36.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015163,0.015163,104,15,0.0,0,0.0,0.0,NaN,NaN,14.0,NaN,1.0,NaN,0.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN
3,0.070621,0,81.0,22.64,1,0.0

In [26]:
train_supplied_missing_catenc.to_csv('train_data_mine.csv')

### prepare test data

In [28]:
unlabeled_supplied = pd.read_csv(unlabeled_data_path)
unlabeled_supplied = pd.DataFrame(data = unlabeled_supplied)

print('Original data shape:\n', unlabeled_supplied.shape, '\n')

#remove columns with meta data (e.g. unique identifiers)
unlabeled_supplied = unlabeled_supplied.drop(columns = meta_cols)

#convert negative numbers to nans 
unlabeled_supplied.loc[~(unlabeled_supplied['pre_icu_los_days'] > 0), 'pre_icu_los_days']=np.nan
unlabeled_supplied.loc[~(unlabeled_supplied['apache_4a_hospital_death_prob'] > 0), 'apache_4a_hospital_death_prob']=np.nan
unlabeled_supplied.loc[~(unlabeled_supplied['apache_4a_icu_death_prob'] > 0), 'apache_4a_icu_death_prob']=np.nan

#get count of missing data per entry
unlabeled_supplied['na_count'] = unlabeled_supplied.isna().sum(axis=1)

#get count for each entry of how many missing for params that have at least 25% difference in missing between yes and no death
unlabeled_supplied['param_25_diff_count'] = unlabeled_supplied[param_25_diff].isna().sum(axis=1)

#remove features that have more than 50% missing data
print(unlabeled_supplied.shape)
unlabeled_supplied_missing = unlabeled_supplied.copy().drop(columns = train_missing)
print(unlabeled_supplied_missing.shape)

#feature mapping
unlabeled_supplied_missing['age_score'] = unlabeled_supplied_missing.apply(lambda row : age_mapping(row['age']), axis = 1)
unlabeled_supplied_missing['chronic_health_score'] = unlabeled_supplied_missing.apply(lambda row : chronic_health_mapping(row, health_conds), axis = 1)
unlabeled_supplied_missing['disease_count'] = unlabeled_supplied_missing[diseases].dropna().sum(axis=1)
unlabeled_supplied_missing['elective_surgery_score'] = unlabeled_supplied_missing.apply(lambda row : elective_surgery_mapping(row['elective_surgery'], row['disease_count']), axis = 1)
unlabeled_supplied_missing['bun_score'] = unlabeled_supplied_missing.apply(lambda row : bun_mapping(row['bun_apache']), axis = 1)
unlabeled_supplied_missing['creatinine_score'] = unlabeled_supplied_missing.apply(lambda row : creatinine_mapping(row['creatinine_apache'], row['arf_apache']), axis = 1)
unlabeled_supplied_missing['GCS_score'] = unlabeled_supplied_missing.apply(lambda row : GCS_mapping(row, gcs_params), axis = 1)
unlabeled_supplied_missing['glucose_score'] = unlabeled_supplied_missing.apply(lambda row : glucose_mapping(row['glucose_apache']), axis = 1)
unlabeled_supplied_missing['heart_rate_score'] = unlabeled_supplied_missing.apply(lambda row : heart_rate_mapping(row['heart_rate_apache']), axis = 1)
unlabeled_supplied_missing['hematocrit_score'] = unlabeled_supplied_missing.apply(lambda row : hematocrit_mapping(row['hematocrit_apache']), axis = 1)
unlabeled_supplied_missing['map_score'] = unlabeled_supplied_missing.apply(lambda row : map_mapping(row['map_apache']), axis = 1)
unlabeled_supplied_missing['resprate_score'] = unlabeled_supplied_missing.apply(lambda row : resprate_mapping(row['resprate_apache'], row['ventilated_apache']), axis = 1)
unlabeled_supplied_missing['sodium_score'] = unlabeled_supplied_missing.apply(lambda row : sodium_mapping(row['sodium_apache']), axis = 1)
unlabeled_supplied_missing['temp_score'] = unlabeled_supplied_missing.apply(lambda row : temp_mapping(row['temp_apache']), axis = 1)
unlabeled_supplied_missing['wbc_score'] = unlabeled_supplied_missing.apply(lambda row : wbc_mapping(row['wbc_apache']), axis = 1)
unlabeled_supplied_missing['d1_calcium_score'] = unlabeled_supplied_missing.apply(lambda row : d1_calcium_mapping(row['d1_calcium_max'], row['d1_calcium_min']), axis = 1)
unlabeled_supplied_missing['d1_platlet_score'] = unlabeled_supplied_missing.apply(lambda row : d1_platlet_mapping(row['d1_platelets_max'], row['d1_platelets_min']), axis = 1)
unlabeled_supplied_missing['d1_potassium_score'] = unlabeled_supplied_missing.apply(lambda row : d1_potassium_mapping(row['d1_potassium_max'], row['d1_potassium_min']), axis = 1)


#encode cat variables
unlabeled_supplied_missing_catenc = enc.transform(unlabeled_supplied_missing)

unlabeled_supplied_missing_catenc.head()

Original data shape:
 (39308, 186) 

(39308, 185)
(39308, 111)


,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,na_count,param_25_diff_count,age_score,chronic_health_score,disease_count,elective_surgery_score,bun_score,creatinine_score,GCS_score,glucose_score,heart_rate_score,hematocrit_score,map_score,resprate_score,sodium_score,temp_score,wbc_score,d1_calcium_score,d1_platlet_score,d1_potassium_score
0,NaN,NaN,56.0,21.102439,0,0.087262,0.084417,180.3,0.087306,0.086385,NaN,0.085168,0.083127,0.104167,68.6,0.072866,0.032178,0,0.0,19.0,0.67,2.0,5.0,0.0,1.0,276.0,105.0,42.0,0.0,57.0,5.0,134.0,36.6,0.0,4.7,74.0,51.0,74.0,51.0,100.0,78.0,86.0,60.0,86.0,60.0,20.0,8.0,100.0,96.0,121.0,79.0,121.0,79.0,37.1,36.6,55.0,51.0,55.0,51.0,88.0,84.0,68.0,60.0,68.0,60.0,18.0,8.0,100.0,98.0,91.0,79.0,91.0,79.0,36.6,36.6,19.0,19.0,9.3,9.3,0.67,0.67,276.0,167.0,24.0,24.0,13.8,13.8,42.0,42.0,223.0,223.0,4.8,4.8,135.0,134.0,4.7,4.7,0.13,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.079018,0.079018,71,15,5.0,0,0.0,0.0,11.0,0.0,8.0,3.0,1.0,0.0,0.0,17.0,1.0,0.0,0.0,0.0,0.0,0.0
1,NaN,NaN,NaN,19.500595,1,0.087262,0.088373,145.0,0.035251,0.037300,NaN,0.085168,0.087495,0.230556,41.0,0.136646,0.172973,1,0.0,NaN,NaN,4.0,6.0,0.0,4.0,NaN,60.0,NaN,0.0,53.0,37.0,NaN,37.0,0.0,NaN,65.0,31.0,65.0,31.0,73.0,60.0,108.0,53.0,108.0,53.0,33.0,16.0,99.0,90.0,148.0,83.0,148.0,83.0,37.7,36.6,50.0,44.0,50.0,44.0,60.0,60.0,80.0,80.0,80.0,80.0,16.0,16.0,94.0,92.0,121.0,96.0,121.0,96.0,36.6,36.6,12.0,12.0,8.0,8.0,0.60,0.60,100.0,100.0,31.0,31.0,9.9,9.9,30.9,30.9,93.0,93.0,4.4,4.4,140.0,140.0,3.0,3.0,0.09,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.079018,0.079018,77,15,NaN,0,0.0,0.0,NaN,NaN,14.0,NaN,0.0,NaN,0.0,9.0,NaN,0.0,NaN,3.0,3.0,0.0
2,NaN,NaN,66.0,32.518596,1,0.078559,0.084417,170.2,0.035251,0.037300,NaN,0.085168,0.075741,0.140972,94.2,0.035363,0.043956,1,0.0,22.0,1.84,2.0,5.0,0.0,1.0,81.0,124.0,39.2,0.0,115.0,31.0,130.0,35.5,0.0,14.8,85.0,57.0,NaN,NaN,117.0,86.0,110.0,71.0,NaN,NaN,28.0,10.0,100.0,95.0,172.0,102.0,NaN,NaN,37.5,35.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,19.0,8.7,8.4,1.84,1.63,117.0,81.0,22.0,19.0,12.7,12.2,40.9,39.2,184.0,181.0,4.7,4.5,135.0,130.0,14.8,10.0,0.03,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.079669,0.108383,94,15,13.0,0,0.0,0.0,11.0,10.0,8.0,0.0,7.0,3.0,5.0,6.0,1.0,2.0,0.0,0.0,0.0,0.0
3,NaN,NaN,64.0,20.798520,0,0.087262,0.084417,182.8,0.035251,0.03

In [35]:
unlabeled_supplied = pd.read_csv(unlabeled_data_path)
unlabeled_supplied = pd.DataFrame(data = unlabeled_supplied)

unlabeled_supplied_missing_catenc['encounter_id'] = unlabeled_supplied['encounter_id']
unlabeled_supplied_missing_catenc.to_csv('test_data_mine.csv')

In [36]:
unlabeled_supplied_missing_catenc.isna().sum()

hospital_id                      37615
hospital_death                   39308
age                               1654
bmi                               1086
elective_surgery                     0
ethnicity                          171
gender                              15
height                             750
hospital_admit_source            11948
icu_admit_source                   115
icu_id                           37615
icu_stay_type                        0
icu_type                             0
pre_icu_los_days                  1812
weight                             764
apache_2_diagnosis                 896
apache_3j_diagnosis                638
apache_post_operative                0
arf_apache                         179
bun_apache                        7016
creatinine_apache                 6896
gcs_eyes_apache                   1163
gcs_motor_apache                  1163
gcs_unable_apache                  546
gcs_verbal_apache                 1163
glucose_apache           

In [ ]:

 'd1_diasbp_max', 'd1_diasbp_min', 'd1_diasbp_noninvasive_max', 'd1_diasbp_noninvasive_min', 
 'd1_sysbp_max', 'd1_sysbp_min', 'd1_sysbp_noninvasive_max', 'd1_sysbp_noninvasive_min',
 'h1_diasbp_max', 'h1_diasbp_min', 'h1_diasbp_noninvasive_max', 'h1_diasbp_noninvasive_min',
 'h1_sysbp_max', 'h1_sysbp_min', 'h1_sysbp_noninvasive_max', 'h1_sysbp_noninvasive_min',

 'd1_heartrate_max', 'd1_heartrate_min', 
 'h1_heartrate_max', 'h1_heartrate_min',
 
 'd1_mbp_max', 'd1_mbp_min', 'd1_mbp_noninvasive_max', 'd1_mbp_noninvasive_min',
 'h1_mbp_max', 'h1_mbp_min', 'h1_mbp_noninvasive_max', 'h1_mbp_noninvasive_min',
 
 'd1_resprate_max', 'd1_resprate_min', 
 'h1_resprate_max', 'h1_resprate_min',
 
 'd1_spo2_max', 'd1_spo2_min',
 'h1_spo2_max', 'h1_spo2_min',
 
 'd1_temp_max', 'd1_temp_min',       
 'h1_temp_max', 'h1_temp_min',
 
 'd1_bun_max', 'd1_bun_min', 
 'd1_calcium_max', 'd1_calcium_min',
 'd1_creatinine_max', 'd1_creatinine_min', 
 'd1_glucose_max', 'd1_glucose_min', 
 'd1_hco3_max', 'd1_hco3_min',
 'd1_hemaglobin_max', 'd1_hemaglobin_min', 
 'd1_hematocrit_max', 'd1_hematocrit_min', 
 'd1_platelets_max', 'd1_platelets_min',
 'd1_potassium_max', 'd1_potassium_min', 
 'd1_sodium_max', 'd1_sodium_min', 
 'd1_wbc_max', 'd1_wbc_min']